In [71]:
import sys
sys.path.append('..')

import os
import shutil
from sqlalchemy import create_engine, text
import geopandas as gpd
import pandas as pd
from geopandas.tools import sjoin
from shapely.geometry import Point, Polygon, MultiPolygon, MultiPoint
from shapely import wkb
import simplekml

from utilities_amigocloud import AmigocloudFunctions

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import API_AMIGOCLOUD_TOKEN_ADM
from config import POSTGRES_UTEA

RUTA_COMPLETA = os.path.join(RUTA_UNIDAD_ONE_DRIVE, RUTA_LOCAL_ONE_DRIVE)

In [72]:
# leer los dos shps de siembra y puntos de planificacion
#PATH_SIEMBRA = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\SIEMBRAS_2.shp'
#PATH_CAT = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\SHP\CATASTRO_S09_MIERCOLES.shp'
PATH_KML = RUTA_UNIDAD_ONE_DRIVE + r'\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN'

#gdf_siembras = gpd.read_file(PATH_SIEMBRA)
#gdf_cat = gpd.read_file(PATH_CAT)

In [73]:
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

def obtener_planificacion_no_procesado():
    engine = obtener_engine()
    try:
        query = """
            SELECT id_os, geom, cod_ca, nom_ca, obs, procesado, producto, dosis, semana_plan
            FROM drones_control_bio.planificacion_ctrl_bio
            WHERE procesado=false;
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def get_catastro():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.catastro_drones
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener la capa de catastro: {e}")
        return gpd.GeoDataFrame()
    return None

def get_siembras():
    engine = obtener_engine()
    try:
        query = f'''
            SELECT * FROM drones_control_bio.siembra
        '''
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener capa de siembras: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_por_id_os(id_os):
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where os = {id_os}
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_sin_planificar():
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where estado IS NULL
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def obtener_parte_diario_planificado():
    engine = obtener_engine()
    try:
        query = f"""
            SELECT * FROM drones_control_bio.parte_diario_ctrl_bio where estado = 'PLANIFICADO'
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error en la consulta: {e}")
        return gpd.GeoDataFrame()
    return None

def filtrar_os_y_quitar_caracteres(gdf, ors):
    # filtrar la semana de interes
    gdf_puntos_select = gdf[gdf['id_os'] == ors]
    gdf_puntos_sem = gdf_puntos_select.copy()
    #quita caracteres especiales
    gdf_puntos_sem['nom_ca'] = gdf_puntos_sem['nom_ca'].str.replace(r'[\r\n\t]', '', regex=True)
    return gdf_puntos_sem

def marcar_como_procesado(id_os):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.planificacion_ctrl_bio
                SET procesado = true
                WHERE id_os = :id_os
            """)
            conn.execute(query, {"id_os": id_os})
            print(f"✔️ id_os {id_os} marcado como procesado.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None

def marcar_como_planificado(id_lote):
    engine = obtener_engine()  # tu función que crea el engine
    try:
        with engine.begin() as conn:
            query = text("""
                UPDATE drones_control_bio.parte_diario_ctrl_bio
                SET estado = 'PLANIFICADO'
                WHERE id = :id_lote
            """)
            conn.execute(query, {"id_lote": id_lote})
            print(f"✔️ id_lote {id_lote} marcado como PLANIFICADO.")
    except Exception as e:
        print(f"❌ Error al actualizar: {e}")
    return None


def crear_puntos_individuales(gdf_fila):
    # Obtener la geometría del único registro
    geom = gdf_fila.iloc[0][gdf_fila.geometry.name]
    # Verificar si es MultiPoint
    if isinstance(geom, MultiPoint):
        # Crear un nuevo registro por cada punto
        registros = []
        for punto in geom.geoms:
            nuevo = gdf_fila.iloc[0].copy()
            nuevo[gdf_fila.geometry.name] = punto
            registros.append(nuevo)
        return gpd.GeoDataFrame(registros, geometry=gdf_fila.geometry.name, crs=gdf_fila.crs)
    else:
        # Si ya es un punto, retornar tal cual
        return gdf_fila.copy()

def definir_lotes_solicitud(gdf_puntos_indi):
    id_os = gdf_puntos_indi.iloc[0]['id_os']
    codigo_ca = gdf_puntos_indi.iloc[0]['cod_ca']
    nombre_ca = gdf_puntos_indi.iloc[0]['nom_ca']
    semana_plan = gdf_puntos_indi.iloc[0]['semana_plan']
    
    # Crear un GeoDataFrame vacío para almacenar los puntos que no intersectan con ningún polígono de siembra
    puntos_indi_solicitado = gdf_puntos_indi[~gdf_puntos_indi.geometry.apply(lambda point: any(gdf_siembras.contains(point)))]
    
    # identifoca los lotes que se intersectan
    gdf_lotes_cat = gpd.sjoin(gdf_cat, puntos_indi_solicitado, how='inner', predicate='intersects')    
    
    # filtra los lotes intersectados en la capa original
    poligonos_intersect_cat = gdf_cat.loc[gdf_cat.index.isin(gdf_lotes_cat.index)]
    
    # copy
    poligonos_intersect_cat = poligonos_intersect_cat.copy()
    poligonos_intersect_cat = poligonos_intersect_cat[['unidad_01', 'unidad_02', 'unidad_03', 'unidad_04', 'unidad_05', 'area', 'soca', 'zona', 'geom']]
    poligonos_intersect_cat["soca"] = poligonos_intersect_cat["soca"].fillna(0)
    # Cambiar el nombre de la columna 'old_name' a 'new_name'
    poligonos_intersect_cat.rename(columns={'zona': 'inst'}, inplace=True)
    # CAMBIAR NOMBRE DATOS DE UNIDAD_03 Y UNDIAD_04 DE LOTES SOLICITADOS A LO INDICADO EN LOS PUNTOS
    poligonos_intersect_cat['unidad_03'] = codigo_ca
    poligonos_intersect_cat['unidad_04'] = nombre_ca
    poligonos_intersect_cat['semana_plan'] = semana_plan
    
    # agrega los campos faltantes
    poligonos_intersect_cat['origen'] = 'SOLICITUD'
    poligonos_intersect_cat['dias'] = 0
    poligonos_intersect_cat['fecha'] = ''
    poligonos_intersect_cat['os'] = id_os
    return poligonos_intersect_cat

def definir_lotes_siembra(gdf_puntos_indi):
    id_os = gdf_puntos_indi.iloc[0]['id_os']
    codigo_ca = gdf_puntos_indi.iloc[0]['cod_ca']
    nombre_ca = gdf_puntos_indi.iloc[0]['nom_ca']
    semana_plan = gdf_puntos_indi.iloc[0]['semana_plan']
    
    # SELECCIONAR LOS LOTES DE SIEMBRAS
    # identifoca los lotes que se intersectan
    gdf_lotes_siem = gpd.sjoin(gdf_siembras, gdf_puntos_indi, how='inner', predicate='intersects')
    # filtra los lotes intersectados en la capa original
    poligonos_intersect_siem = gdf_siembras.loc[gdf_siembras.index.isin(gdf_lotes_siem.index)]
    # copy
    poligonos_intersect_siem = poligonos_intersect_siem.copy()
    # agregar campo de semana_planificacion
    poligonos_intersect_siem['os'] = id_os
    # crea el campo soca, y coloca 0 o 1 dependiendo del origen
    poligonos_intersect_siem['soca'] = poligonos_intersect_siem['origen'].apply(lambda x: 1 if 'SIEMBRA 2024' in x else 0)

    # agrega los campos faltantes
    poligonos_intersect_siem['semana_plan'] = semana_plan
    
    # ordena los campos
    poligonos_intersect_siem = poligonos_intersect_siem[['unidad_01', 'unidad_02', 'unidad_03', 'unidad_04', 'unidad_05', 'area', 'soca', 'inst',
           'geom', 'origen', 'dias', 'fecha', 'os', 'semana_plan']]
    return poligonos_intersect_siem

def agregar_campos_faltantes(gdf_plan):
    gdf_plan['producto'] = ''
    gdf_plan['dosis'] = 0.0
    gdf_plan['id_labor'] = ''
    gdf_plan['fecha'] = ''
    gdf_plan['semana'] = 0
    gdf_plan['num_lib'] = 0
    gdf_plan['hora_ini'] = ''
    gdf_plan['hora_fin'] = ''
    gdf_plan['temp'] = 0
    gdf_plan['viento'] = 0
    gdf_plan['humedad'] = 0
    gdf_plan['cod_dron'] = ''
    gdf_plan['piloto_1'] = ''
    gdf_plan['piloto_2'] = ''
    gdf_plan['obs'] = ''
    gdf_plan['id_log'] = 0
    return gdf_plan

def convertir_a_multipolygon(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    return geometry

def convertir_a_wkb(polygon):
    wkb_data = wkb.dumps(polygon, hex=True)
    return wkb_data

def cargar_a_amigocloud(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['dias'] = gdf_pla_gral['dias'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    gdf_pla_gral['semana_plan'] = gdf_pla_gral['semana_plan'].astype(int)
    
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_345601 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, etiqueta, producto, dosis, cantidad, semana_plan, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', '{str(round(row['area'],2)) + 'ha - ' + str(round(row['producto_cantidad'],2)) + 'ml'}', '{row['producto_plan']}', {row['producto_dosis']}, {row['producto_cantidad']}, {row['semana_plan']}, ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
            """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        marcar_como_planificado(row['id'])
    return None

def cargar_a_amigocloud_semana_siguiente(gdf):
    # repreyectar a WGS84
    gdf_pla_gral = gdf.to_crs(epsg=4326)
    # convertir poligonos a multipoligonos
    gdf_pla_gral['geom'] = gdf_pla_gral['geom'].apply(convertir_a_multipolygon)
    
    gdf_pla_gral['unidad_01'] = gdf_pla_gral['unidad_01'].astype(int)
    gdf_pla_gral['unidad_03'] = gdf_pla_gral['unidad_03'].astype(int)
    gdf_pla_gral['dias'] = gdf_pla_gral['dias'].astype(int)
    gdf_pla_gral['os'] = gdf_pla_gral['os'].astype(int)
    gdf_pla_gral['soca'] = gdf_pla_gral['soca'].astype(int)
    gdf_pla_gral['id'] = gdf_pla_gral['id'].astype(int)
    gdf_pla_gral['inst'] = gdf_pla_gral['inst'].astype(int)
    
    
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf_pla_gral.iterrows():
        wkb_hex = convertir_a_wkb(row['geom'])
        insert_sql = f"""
            INSERT INTO dataset_381625 (id, unidad_01, unidad_02, unidad_03, unidad_04, unidad_05, area, origen, dias, os, etiqueta, producto, dosis, cantidad, semana_plan, geometry)
            VALUES ({row['id']}, {row['unidad_01']}, '{row['unidad_02']}', {row['unidad_03']}, '{row['unidad_04']}', '{row['unidad_05']}', {row['area']}, '{row['origen']}', '{row['dias']}', '{row['os']}', '{str(round(row['area'],2)) + 'ha - ' + str(round(row['producto_cantidad'],2)) + 'ml'}', '{row['producto_plan']}', {row['producto_dosis']}, {row['producto_cantidad']}, {row['semana_plan']}, ST_SetSRID(ST_GeomFromWKB('\\x{wkb_hex}'), 4326));
            """
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f"Lotes cargado proxima semana ID: {row['id']}")
        # marcar_como_planificado(row['id'])
    return None

def cargar_a_amigocloud_lista_canhero_notif(gdf):
    # recorrer el gdf de lotes y cargarlo a amigocloud
    id_proyecto = 33457
    for index, row in gdf.iterrows():
        cod = row['unidad_03']
        nom = row['unidad_04']
        cod_canhero = str(cod) + ' / ' + nom
        insert_sql = f"insert into dataset_354656 (cod_canhero) values('{cod_canhero}')"
        query_sql = {'query': insert_sql}
        resultado_post = amigocloud.ejecutar_query_sql(id_proyecto, insert_sql, 'post')
        print(f"Se ha registrado nuevo cañero para notificacion: {cod_canhero}")
        # marcar_como_planificado(row['id'])
    return None

# verifica si en la dataframe existen nombres de lotes repetidos
def validar_lotes_repetidos(gdf_plan):
    cods_props = list(set(gdf_plan['unidad_01']))
    for i in cods_props:
        prop = gdf_plan[gdf_plan['unidad_01'] == i]
        dup = prop[prop['unidad_05'].duplicated(keep=False)]
        if prop['unidad_05'].duplicated().any():
            print('Lotes repedidos:', list(set(dup['unidad_05'])))
            return True
        else:
            return False
    return None

def crear_kmls(gdf_lotes):
    lista_cods = list(set(gdf_lotes['unidad_01']))
    for i in lista_cods:
        prop = gdf_lotes[gdf_lotes['unidad_01'] == i]
        prop_wgs = prop.to_crs(epsg=4326)
        output_folder = PATH_KML + '\\' + str(i)
        print(output_folder)
        
        if os.path.exists(output_folder):
            shutil.rmtree(output_folder)  # Elimina toda la carpeta y su contenido
        os.makedirs(output_folder)  # La vuelve a crear vacía
        
        # Recorrer el GeoDataFrame
        for idx, row in prop_wgs.iterrows():
            # Crear un nuevo documento KML
            kml_doc = simplekml.Kml()
            # Obtener el polígono y el nombre
            polygon = row['geom']
            name = row['unidad_05']
            # Convertir el polígono en una lista de coordenadas
            if polygon.geom_type == "Polygon":
                coords = [(x, y) for x, y in zip(*polygon.exterior.xy)]
            elif polygon.geom_type == "MultiPolygon":
            # puedes iterar si quieres más de uno, aquí tomamos el primero
                first_poly = list(polygon.geoms)[0]
                coords = [(x, y) for x, y in zip(*first_poly.exterior.xy)]
            else:
                print(f"❌ Geometría no compatible: {polygon.geom_type}")
                continue  # salta este registro
            # Añadir el polígono al documento KML
            pol = kml_doc.newpolygon(name=name, outerboundaryis=coords)
            # Guardar el archivo KML
            file_name = f"{name}.kml"
            output_path = os.path.join(output_folder, file_name)
            kml_doc.save(output_path)
    return None

In [74]:
amigocloud = AmigocloudFunctions(token=API_AMIGOCLOUD_TOKEN_ADM)
amigocloud

# ACTUALIZAR SHP PLANIFICACION

In [77]:
gdf_siembras = get_siembras()
gdf_cat = get_catastro()

In [78]:
gdf_puntos = obtener_planificacion_no_procesado()

In [79]:
gdf_puntos

,id_os,geom,cod_ca,nom_ca,obs,procesado,producto,dosis,semana_plan
0,473,"MULTIPOINT (491330.619 8103489.565, 490615.329...",41562,SUAREZ SALAS ERWIN RONALD,None,False,trichogramma,6.0,52
1,474,MULTIPOINT (488467.939 8100710.028),13062,SOTO ARCE BACILIO,None,False,trichogramma,6.0,52
2,475,"MULTIPOINT (486485.800 8103044.004, 487198.177...",10697,PAZ REA JAIME EDUARDO,None,False,trichogramma,6.0,52
3,476,"MULTIPOINT (458539.947 8096313.843, 460160.853...",11860,ROCA SERRANO ARMANDO,None,False,trichogramma,6.0,52
4,480,"MULTIPOINT (491602.888 8088803.507, 491390.324...",388,AGROPECUARIA MARIANA S.R.L.,None,False,trichogramma,6.0,52
5,481,"MULTIPOINT (485766.175 8087333.004, 485635.883...",18088,FRERKING ORTIZ RICARDO,None,False,trichogramma,6.0,52
6,482,"MULTIPOINT (482398.138 8088771.778, 482403.650...",388,AGROPECUARIA MARIANA S.R.L.,None,False,trichogramma,6.0,52
7,485,"MULTIPOINT (484561.187 8095000.971, 484185.897...",4974,GUTIERREZ GIL MARCIAL,None,False,trichogramma,6.0,52
8,486,MULTIPOINT (481834.460 8089828.458),14760,VARGAS PINTO RENE,None,False,trichogramma,6.0,52
9,497,"MULTIPOINT (500045.045 8096955.841, 500165.089...",41594,AGROPECUARIA CAMPO DULCE S.R.L.,None,False,trichogramma,6.0,52


In [80]:
for i, row in gdf_puntos.iterrows():
    ors = row['id_os']
    produc_plan = row['producto']
    produc_dosis = row['dosis']
    
    puntos = filtrar_os_y_quitar_caracteres(gdf_puntos, ors)
    multipoint_gdf = puntos.copy()
    puntos_individuales = crear_puntos_individuales(multipoint_gdf)
    
    lotes_solicidud = definir_lotes_solicitud(puntos_individuales)
    lotes_siembra = definir_lotes_siembra(puntos_individuales)
    
    # verificar si algun dataframe esta vacio
    dfs = [df for df in [lotes_solicidud, lotes_siembra] if not df.empty]
    # concatena los dfs no vacios
    concat_plan = pd.concat(dfs, axis=0, ignore_index=True)
    
    plan = agregar_campos_faltantes(concat_plan)
    #RECALCULAR EL AREA
    plan['area'] = plan.geom.area / 10000

    plan['producto_plan'] = produc_plan
    plan['producto_dosis'] = produc_dosis
    plan['producto_cantidad'] = plan['area'] * plan['producto_dosis']
    
    #plan.rename(columns={'geom': 'geom'}, inplace=True)
    #plan = plan.set_geometry("geom")
    plan['unidad_01'] = plan['unidad_01'].astype(int)
    plan['unidad_03'] = plan['unidad_03'].astype(int)
    plan['dias'] = plan['dias'].astype(int)
    plan['os'] = plan['os'].astype(int)
    plan['soca'] = plan['soca'].astype(int)
    plan['inst'] = plan['inst'].astype(int)

    # validas que la planificacion no tenga lotes con el mismo nombre
    if validar_lotes_repetidos(plan):
        print(f"ERROR: Existen lotes con nombre repetido en la propiedad: {plan.iloc[0]['unidad_01']}")
        continue
    #crear_kmls(plan)    
    plan.to_postgis("parte_diario_ctrl_bio", obtener_engine(), schema="drones_control_bio", if_exists="append")
    
    marcar_como_procesado(ors)

✔️ id_os 473 marcado como procesado.
✔️ id_os 474 marcado como procesado.
✔️ id_os 475 marcado como procesado.
✔️ id_os 476 marcado como procesado.
✔️ id_os 480 marcado como procesado.
✔️ id_os 481 marcado como procesado.
✔️ id_os 482 marcado como procesado.
✔️ id_os 485 marcado como procesado.
✔️ id_os 486 marcado como procesado.
✔️ id_os 497 marcado como procesado.
✔️ id_os 498 marcado como procesado.
✔️ id_os 499 marcado como procesado.
✔️ id_os 500 marcado como procesado.
✔️ id_os 483 marcado como procesado.
✔️ id_os 484 marcado como procesado.
✔️ id_os 487 marcado como procesado.
✔️ id_os 488 marcado como procesado.
✔️ id_os 489 marcado como procesado.
✔️ id_os 490 marcado como procesado.
✔️ id_os 491 marcado como procesado.
✔️ id_os 492 marcado como procesado.
✔️ id_os 493 marcado como procesado.
✔️ id_os 494 marcado como procesado.
✔️ id_os 495 marcado como procesado.
✔️ id_os 496 marcado como procesado.


## Cargar a AmigoCloud los lotes sin planificar

In [82]:
lotes_sin_planificar = obtener_parte_diario_sin_planificar()
print('Lotes sin planificar:', len(lotes_sin_planificar))

Lotes sin planificar: 294


In [83]:
lotes_sin_planificar['area'].sum()

2645.053438051988

In [84]:
cargar_a_amigocloud(lotes_sin_planificar)

✔️ id_lote 4385 marcado como PLANIFICADO.
✔️ id_lote 4386 marcado como PLANIFICADO.
✔️ id_lote 4387 marcado como PLANIFICADO.
✔️ id_lote 4388 marcado como PLANIFICADO.
✔️ id_lote 4389 marcado como PLANIFICADO.
✔️ id_lote 4390 marcado como PLANIFICADO.
✔️ id_lote 4391 marcado como PLANIFICADO.
✔️ id_lote 4392 marcado como PLANIFICADO.
✔️ id_lote 4393 marcado como PLANIFICADO.
✔️ id_lote 4394 marcado como PLANIFICADO.
✔️ id_lote 4048 marcado como PLANIFICADO.
✔️ id_lote 4167 marcado como PLANIFICADO.
✔️ id_lote 4081 marcado como PLANIFICADO.
✔️ id_lote 4166 marcado como PLANIFICADO.
✔️ id_lote 3690 marcado como PLANIFICADO.
✔️ id_lote 3826 marcado como PLANIFICADO.
✔️ id_lote 4109 marcado como PLANIFICADO.
✔️ id_lote 3688 marcado como PLANIFICADO.
✔️ id_lote 4216 marcado como PLANIFICADO.
✔️ id_lote 4217 marcado como PLANIFICADO.
✔️ id_lote 4028 marcado como PLANIFICADO.
✔️ id_lote 4037 marcado como PLANIFICADO.
✔️ id_lote 4128 marcado como PLANIFICADO.
✔️ id_lote 4049 marcado como PLANI

In [88]:
lotes_planificados = obtener_parte_diario_planificado()

In [91]:
lotes_planificados["area"].sum()

2645.0534380519875

In [92]:
lotes_planificados = obtener_parte_diario_planificado()
lotes_planificados = lotes_planificados[['unidad_03', 'unidad_04']]
lotes_planificados = lotes_planificados.drop_duplicates(subset=['unidad_03'])
lotes_planificados

,unidad_03,unidad_04
0,42239,CORDOVA OLGUIN LEONARDO
5,5006,GARCIA MARISCAL MILTON FELIX
8,42266,GARCIA PETRONILA MARISCAL VDA. DE
10,14562,VACA HURTADO JULIO ERNESTO
11,41599,LOPEZ TURIHUANO JUAN CARLOS
12,41531,GUZMAN ALMANZA WILMER
15,12162,ROCA REYES MARCOS
16,3323,TREPP CARRASCO RUDIGER
17,4967,GUZMAN ALMANZA WILLAN
18,566,ARNEZ DURAN PEDRO


In [93]:
cargar_a_amigocloud_lista_canhero_notif(lotes_planificados)

Se ha registrado nuevo cañero para notificacion: 42239 / CORDOVA OLGUIN LEONARDO
Se ha registrado nuevo cañero para notificacion: 5006 / GARCIA MARISCAL MILTON FELIX
Se ha registrado nuevo cañero para notificacion: 42266 / GARCIA PETRONILA MARISCAL VDA. DE
Se ha registrado nuevo cañero para notificacion: 14562 / VACA HURTADO JULIO ERNESTO
Se ha registrado nuevo cañero para notificacion: 41599 / LOPEZ TURIHUANO JUAN CARLOS
Se ha registrado nuevo cañero para notificacion: 41531 / GUZMAN ALMANZA WILMER
Se ha registrado nuevo cañero para notificacion: 12162 / ROCA REYES MARCOS
Se ha registrado nuevo cañero para notificacion: 3323 / TREPP CARRASCO RUDIGER
Se ha registrado nuevo cañero para notificacion: 4967 / GUZMAN ALMANZA WILLAN
Se ha registrado nuevo cañero para notificacion: 566 / ARNEZ DURAN PEDRO 
Se ha registrado nuevo cañero para notificacion: 6578 / JALDIN ARREDONDO JOSE LUIS
Se ha registrado nuevo cañero para notificacion: 11860 / ROCA SERRANO ARMANDO
Se ha registrado nuevo cañer

## Cargar a otro dataset los lotes sin planificar

In [33]:
cargar_a_amigocloud_semana_siguiente(lotes_sin_planificar)

Lotes cargado proxima semana ID: 3618
Lotes cargado proxima semana ID: 3619
Lotes cargado proxima semana ID: 3620
Lotes cargado proxima semana ID: 3621
Lotes cargado proxima semana ID: 3622
Lotes cargado proxima semana ID: 3623
Lotes cargado proxima semana ID: 3624
Lotes cargado proxima semana ID: 3625
Lotes cargado proxima semana ID: 3626
Lotes cargado proxima semana ID: 3627
Lotes cargado proxima semana ID: 3628
Lotes cargado proxima semana ID: 3629
Lotes cargado proxima semana ID: 3630
Lotes cargado proxima semana ID: 3631
Lotes cargado proxima semana ID: 3632
Lotes cargado proxima semana ID: 3633
Lotes cargado proxima semana ID: 3634
Lotes cargado proxima semana ID: 3635
Lotes cargado proxima semana ID: 3636
Lotes cargado proxima semana ID: 3637
Lotes cargado proxima semana ID: 3638
Lotes cargado proxima semana ID: 3639
Lotes cargado proxima semana ID: 3640
Lotes cargado proxima semana ID: 3641
Lotes cargado proxima semana ID: 3642
Lotes cargado proxima semana ID: 3643
Lotes cargad

## Crear KML de lotes planificados

In [94]:
lotes_planificados = obtener_parte_diario_planificado()

In [95]:
len(lotes_planificados)

294

In [96]:
lotes_planificados["area"].sum()

2645.0534380519875

In [97]:
crear_kmls(lotes_planificados)

G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\1
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\1411
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\772
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\773
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\259
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\391
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\2308
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\2307
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA\Trichogramma\2025\KML_PLAN\1290
G:\\Ingenio Azucarero Guabira S.A\UTEA - SEMANAL - EQUIPO AVIACION UTEA